# Training

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b", quantization_config=bnb_config)

if torch.cuda.is_available():
    model = model.to("cuda")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
from datasets import load_dataset

ds = load_dataset("NilanE/ParallelFiction-Ja_En-100k")
split = ds['train'].train_test_split(test_size=0.01, seed=42)
ds = split["test"]

In [3]:
def format_instruction(sample):
    genres_list = sample['meta']['novelupdates']['genres']
    
    # Example: "Adventure, Fantasy, Shounen"
    genres_str = ", ".join(genres_list)
    
    # We ask the model to write a story including these specific genres.
    instruction = f"「{genres_str}」のジャンルが含まれるストーリーを書いて。"
    
    # Standard Rinna Format: ユーザー: {instruction}\nシステム: {response}
    prompt = f"ユーザー: {instruction}\nシステム: {sample['src']}"
    
    return {"text": prompt}

# Apply to the dataset
formatted_ds = ds.map(format_instruction)

In [4]:
formatted_ds['text'][:3]

['ユーザー: 「Romance, Slice of Life」のジャンルが含まれるストーリーを書いて。\nシステム: 真面目な話をこれからすると思うと、自然と真面目な顔つきになる。\n姫川もさっきから、俺の事を直視しながら、どう切り出すか考えているのだろう。\n部屋には時計の秒針がコチコチ音を鳴らすだけで、他の音は聞こえなかった。\n静寂の中、ついに意を決し姫川は語り始める。\n「実は......」\n『ぐぅぅぅ~~~』\nこれは、姫川の作戦か? 俺は思わず頬を吊り上げ、にやけてしまった。\n「ひ、姫川。今の音は?」\n姫川は頬を赤くしながら下を向いてしまい、返事はない。\n少し肩が震えているので、恐らく恥ずかしさを我慢しているのだろう。\n「い、今の音は何でもありません。き、気にしないでください」\n「ちなみに、姫川が最後に食べたのはいつだ?」\n今だに頬を赤くしている姫川は、いつもより若干幼く見える。\n同い年だし、普段は俺よりもよっぽど大人っぽい仕草を見せられているが、今は後輩のように見える。\n「昨夜食べたっきりで、今日はまだ何も......」\n「そっか。話は後だ、ちょっと待ってろ」\n俺は、姫川をリビングに放置し台所に向かう。\n台所から手に持ったタウン誌を姫川に放り投げる。反射的に受け取った姫川はタウン誌片手に俺の方を見る。\n「今月のタウン誌。少し時間かかるからそれでも読んで暇つぶしててくれ。あ、そこの陰にコンセントあるから充電とか適当にしてていいぞ」\nそして俺は台所に行き、いつもつけている前掛けを装備する。\nそこまで料理は得意ではないが、それなりにできるはず。社長令嬢が食べていた食事と比べればお粗末なものができるだろう。\nだが、何も食べないよりはましだ。\n昨日作っておいた調理済みの料理を冷蔵庫から出し、皿に盛りつける。\nあとはご飯と、汁物。汁物は作り置きが無いので適当に作る。\n火の通りやすい野菜にワカメでいいか。\nそして、最後に魔法をかける。おそらく誰しもが使った事のある魔法。そう、レンチンだ。\n盛ったおかずに、ご飯をレンジに入れチンする。\n汁物だけは今作ったので温かさそのまま。\n――チン!\nレンジから取り出した料理をトレイにおき、箸をそえ台所を後にする。\n俺は雑誌を読んでる姫川の目の前に、適当に準備したご飯を置いていく。

In [5]:
split_ds = formatted_ds.train_test_split(test_size=0.1, seed=10) #seeded so the split is not random

train_set = split_ds['train']
eval_set = split_ds['test']

In [6]:
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments

In [7]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [8]:
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(65536, 2816)
    (emb_dropout): Dropout(p=0.1, inplace=False)
    (layers): ModuleList(
      (0-35): 36 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2816,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.1, inplace=False)
        (post_mlp_dropout): Dropout(p=0.1, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear4bit(in_features=2816, out_features=8448, bias=True)
          (dense): Linear4bit(in_features=2816, out_features=2816, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear4bit(in_features=2816, out_features=11264, bias=True)
          (dense_4h_to_h): Linear4bit(in_features=11264, out_features=2816, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((2

In [11]:
training_args = TrainingArguments(
    output_dir="./lora-fine-tuned-model",
    num_train_epochs=1,              # Specifically setting 1 epoch
    per_device_train_batch_size=2,   # Adjust based on GPU VRAM
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_strategy="epoch"            # Save after the epoch is done
)


def formatting_prompts_func(example):
    return example['text'] # The trainer expects a list of strings


trainer = SFTTrainer(
    model=model,
    train_dataset=train_set,      
    processing_class=tokenizer,          
    args=training_args,           
    peft_config=lora_config,      
    formatting_func=formatting_prompts_func, 
)


trainer.train()

c:\Users\A\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\Users\A\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Step,Training Loss
10,9.793900
20,7.915200
30,7.429100
40,6.964800
50,6.365500
60,5.398000
70,4.392000
80,3.957500
90,3.649700
100,3.555800


TrainOutput(global_step=477, training_loss=3.867783244550853, metrics={'train_runtime': 29871.4959, 'train_samples_per_second': 0.032, 'train_steps_per_second': 0.016, 'total_flos': 2.1252751603623936e+16, 'train_loss': 3.867783244550853})

# Evaluation

In [10]:
# Note:
# Cap at 512 or 1024 tokens next time instead of whole chapters cause this took way too long 
from peft import PeftModel

lora_checkpoint_path = "./lora-fine-tuned-model/checkpoint-477"
model = PeftModel.from_pretrained(model, lora_checkpoint_path)
model = model.merge_and_unload()

c:\Users\A\AppData\Local\Programs\Python\Python311\Lib\site-packages\peft\tuners\lora\bnb.py:397: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# One prompt to test
text = "「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。"
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        min_new_tokens=100,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。 また、お酒のラベルも書いて。 これまた、とってもお上手。 しかも、ちゃんと、お酒の味もわかって、お上手。 で、このあと、「あのラベルってさ」 って、お母さまに見せたりしたら、 「えーーー! あの子、今年、お酒のラベルまで、書いてたんだよー」 って、お母さまは、かなり、お気に召したよう。 この


In [18]:

target_length = 2000
max_new_tokens_per_loop = 512 
story = "「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。"

# Encode the initial prompt
input_ids = tokenizer.encode(story, return_tensors="pt").to("cuda")

print("--- Starting the Story ---")
print(story, end="")

while input_ids.shape[1] < target_length:
    # Generate the next segment
    output_sequences = model.generate(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens_per_loop,
        do_sample=True,
        top_p=0.95,
        temperature=0.8,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

    new_tokens = output_sequences[:, input_ids.shape[1]:]
    
    new_text = tokenizer.decode(new_tokens[0], skip_special_tokens=True)
    print(new_text, end="", flush=True)

    input_ids = output_sequences

    # Safety break if the model generates an eos token
    if tokenizer.eos_token_id in new_tokens:
        print("\n--- Model reached end of story. ---")
        break

--- Starting the Story ---
「Shounen, Adventure」のジャンルが含まれるストーリーを書いて。「(今日は)自分の投球をしっかりできたし、先制点を取ることができたのでよかったです」
「試合前に、みんなから『絶対抑えろ』って言われてました。でも10回で2アウトになって、1対0だったので、あまり気持ち的には変わらないんですけどね(笑)」
「(逆転された5回裏に)あそこから粘れたというのは、自分にも自信になりますけど、相手のピッチャーにもすごく大きなものがあったと思います。そこを取り返してやるという強い思いが伝わってきてたので、それが最後の1点がこっちに来たんだろうなと。やっぱり悔しいですよね」
「これからもっと厳しい練習になっていくと思うんですけれども、そういう意味では、自分がこうやって経験させてもらっていることに感謝しながら、とにかく頑張っていきたいと思います」
「今年は結果が出なくて、悔しくて、この1年何をやってきたのかと反省するところもあったんですけれども、本当にいろんな人が支えてくれて、一緒に戦ってきてくれたことがすごく大きかったと思います。そういう方々に恩返しできるように頑張りたいとおもいます」
「今日みたいな戦い方をしてくれると、これからもやりやすいんじゃないかと思いましたね。なかなか、ああいう戦い方は見たことがないんで、楽しかったですね。それくらいしか言えないです」
--- Model reached end of story. ---
